# Panel library examples

Creating dashboards
[https://panel.pyviz.org/index.html]

In [ ]:
import io
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import param
import panel as pn

css = '''
.widget-box {
  color: #ff0000;
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
}
'''
pn.extension(raw_css=[css])

In [ ]:
pn.__version__

In [ ]:

tap = hv.streams.PointerX(x=0)


def hvplot2(df, frequency, **kwargs):
    plot = df.hvplot(width=500, padding=(0, 0.1))
    tap.source = plot

    def unit_circle(x):
        cx       = np.cos(x*frequency)
        sx       = np.sin(x*frequency)
        circle   = hv.Path([ hv.Ellipse(0,0,2),  [(-1,0), (1,0)],  [(0,-1), (0,1)]]).opts(color='black')
        triangle = hv.Path([ [(0,0), (cx,sx)],   [(0,0), (cx,0)],  [(cx,0), (cx,sx)]]).opts(color='red', line_width=2)
        labels   = hv.Labels([(cx/2, 0, '%.2f' % cx), (cx, sx/2., '%.2f' % sx)])
        labels   = labels.opts(padding=0.1, xaxis=None, yaxis=None, text_baseline='bottom')
        return (circle * triangle * labels)

    vline  = hv.DynamicMap(hv.VLine,    streams=[tap]).opts(color='black')
    circle = hv.DynamicMap(unit_circle, streams=[tap])

    return (plot * vline + circle).opts(toolbar="right")

def sine(frequency=1.0, amplitude=1.0, n=200, view_fn=hvplot2):
    xs = np.arange(n)/n*20.0
    ys = amplitude*np.sin(frequency*xs)
    df = pd.DataFrame(dict(y=ys), index=xs)
    return view_fn(df, frequency=frequency, amplitude=amplitude, n=n)


unit_curve = pn.interact(sine, view_fn=hvplot2, n=(1, 200), frequency=(0, 10.))

pn.Column(pn.Row('# The Unit Circle', pn.Spacer(width=45), unit_curve[0][0], unit_curve[0][2]),
          unit_curve[1],css_classes=['widget-box'])

# Widgets

## Text input

In [ ]:
widget = pn.widgets.TextInput(name='A widget', value='A string',css_classes=['widget-box'])
#widget.background = 'black'
widget

In [ ]:
widget.value

## Autocompleter

In [ ]:
opt_list = ['fred','george','harold'] +['parameter %i' % n for n in range(10)]
com = pn.widgets.AutocompleteInput(name='Autocompleter',options=opt_list,placeholder='Enter text here',
                                   css_classes=['widget-box'])
com

In [ ]:
column = pn.Column('# A title', com,widget)
column

## File selector

In [ ]:
file_selector = pn.widgets.FileInput(name='Load file',css_classes=['widget-box'])
file_selector

In [ ]:
file_selector.mime_type

In [ ]:
file_selector.value.decode().splitlines()

In [ ]:
file_selector.value

In [ ]:
file_selector.core.filename

# Read Excel data using file selector

In [ ]:
file_selector

In [ ]:
file_selector.mime_type

In [ ]:
excel_file = io.BytesIO(file_selector.value)

In [ ]:
df = pd.read_excel(excel_file)

In [ ]:
df

# Param based GUIs

In [ ]:
class Example(param.Parameterized):
    """An example Parameterized class"""

    boolean                 = param.Boolean(True, doc="A sample Boolean parameter")
    boolean2                = param.Boolean(False, doc="This boolean is going to be a button")
    
    color                   = param.Color()
    select_string           = param.ObjectSelector(default="yellow", objects=["red", "yellow", "green"])
    select_fn               = param.ObjectSelector(default=list,objects=[list, set, dict])
    int_list                = param.ListSelector(default=[3, 5], objects=[1, 3, 5, 7, 9], precedence=0.5)
    single_file             = param.FileSelector(path='*.ipy*', precedence=0.5)
    multiple_files          = param.MultiFileSelector(path='*.ipy*', precedence=0.5)

In [ ]:


fs = Example()
pn.Param(fs,widgets={'boolean2':{'type':pn.widgets.Toggle,'name':'Button','button_type':'success'}})
#pn.Column(fs.param)



In [ ]:
fs.single_file

## File selector
Using param

In [ ]:
class FileSelector(param.Parameterized):
    """An example Parameterized File selector class"""

    title        = param.String(default="Select File to open", doc="A string")
    single_file  = param.FileSelector(path='*.ipy*') 
    button       = param.Boolean(False, doc="This boolean is going to be a button")
    status       = param.String(default="Nothing", doc="A string")
    
    @param.depends('button',watch=True)
    def open_file(self):
        # This doesn't work
        if self.button:
            self.status = 'Open file: %s' % self.single_file
        
    
fs = FileSelector()
pn.Param(fs,widgets={'button':{'type':pn.widgets.Toggle,'name':'Open','button_type':'success'}})

# END